In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt

import file_ext_search as fes

import asd
import trace_proc

idx_file2 = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\PS3SLF_2021-09-17T110351Z_00047728.asd.acf.idx'
acf_file2 = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\PS3SLF_2021-09-17T110351Z_00047728.asd.acf'


idx_file = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\Prof2_abp56_Gd\PS3SLF_2024-07-07T174231Z_07793648.asd.acf.idx'
acf_file = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\Prof2_abp56_Gd\PS3SLF_2024-07-07T174231Z_07793648.asd.acf'

idx_files_2 = [idx_file2, idx_file]

datapath = 'D:\ABP49_Processing'
idx_files = fes.file_ext_search('.idx', datapath)

Searching *.idx files in directory:D:\ABP49_Processing


In [60]:
save_to = r'C:\Data\test\ABP49_SLF.csv'

for idx_file in idx_files:
    pos_strings = trace_proc.get_pos_acf(idx_file)
    
    
    if os.path.exists(save_to):
        with open(save_to, 'a+') as f9:
            for line in pos_strings:
                f9.write(f'{line}\n')
    else:
        with open(save_to, 'w') as f9:
            f9.write('num,acf,asd,ISOdatetime,POSIXsec,lat,lon\n')
            
            for line in pos_strings:
                f9.write(f'{line}\n')

In [50]:
print(pos_strings)

['PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110351Z_00047728.asd,2021-09-17T11:03:49.027000+00:00,1631876629.0270002,54.90180141684311,19.83370757147687', 'PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110353Z_00047792.asd,2021-09-17T11:03:50.027000+00:00,1631876630.027,54.90181860557696,19.833719030632775', 'PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110353Z_00047792.asd,2021-09-17T11:03:51.027000+00:00,1631876631.027,54.90184152388876,19.83373621936663', 'PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110353Z_00047792.asd,2021-09-17T11:03:52.027000+00:00,1631876632.027,54.90185871262262,19.83375340810048', 'PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110354Z_00047888.asd,2021-09-17T11:03:52.027000+00:00,1631876632.0270002,54.90185871262262,19.83375340810048', 'PS3SLF_2021-09-17T110351Z_00047728.asd.acf,PS3SLF_2021-09-17T110355Z_00047952.asd,2021-09-17T11:03:53.027000+00:00,1631876633.027,54.90188163093

In [10]:

for file in idx_files_2[1:]: 
    acf_file = file[:-4]   
    asd_obj_list = asd.ASDfile.create_from_idx_file(file)

    with open(acf_file, 'rb') as f1:
        buffer = f1.read()    
        
    traces = []
    start_times = []

    delay = 50  # ms
    trace_len = 250  # ms

    num_of_traces = 0
    dt = 0.082
    
    heave_array = None
    
    
    # for obj in asd_obj_list[100:101]:
    for obj in asd_obj_list[:]:
        if num_of_traces > 1800:
            break
        else:
            asd.parse_xml_header(obj, buffer)
            asd.parse_bin_header(obj, buffer)

            for sounding in obj.soundings[:]:
                ampls = trace_proc.proc_trace(sounding, obj, tracelen=trace_len, delay=delay)
                dt = sounding.ampl_scan_interval*1000
                # trace.dt = sounding.ampl_scan_interval*1000  # in ms
                # trace.data = ampls
                num_of_traces += 1
                
                traces.append(ampls)


## SEGY

In [8]:
import numpy as np

import matplotlib.pyplot as plt
import segyio


sample_no = len(traces[0])

class MySpec(object):
    def __init__(self):
        self.iline = 189  # default value for segyio
        self.xline = 193  # default value for segyio
        self.tracecount = 0
        self.samples = []  # sample times, a list
        self.ext_headers = 0
        self.format = 5  # 4-byte IEEE float
        self.endian = 'big'
        
path = r'C:\Data\test\mytest09_new_HeaveAttitude.sgy'

myspec = MySpec()

myspec.tracecount = len(traces)
myspec.samples = np.arange(delay,delay+trace_len+dt,dt)

with segyio.create(path, myspec) as f5:
    for num, trace in enumerate(traces):
        f5.trace[num] = trace

    # print(f5.header[0])
    
    

In [83]:
import numpy as np

import matplotlib.pyplot as plt
import segyio


class MySpec(object):
    def __init__(self):
        self.iline = 189  # default value for segyio
        self.xline = 193  # default value for segyio
        self.tracecount = 0
        self.samples = []  # sample times, a list
        self.ext_headers = 0
        self.format = 5  # 4-byte IEEE float
        self.endian = 'big'

def plot_rawtraces(raw_traces):
    clip = 1e+3
    vmin, vmax = -clip, clip

    # Figure
    figsize=(20, 20)
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=figsize, facecolor='w', edgecolor='k',
                        squeeze=False,
                        sharex=True)
    axs = axs.ravel()
    im = axs[0].imshow(raw_traces.T, cmap=plt.cm.seismic, vmin=vmin, vmax=vmax)
    plt.show()


path = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\Profile-0_W1_SLF2109171103_LL_car.sgy'


with segyio.open(filename=path, mode="r",
                            strict = False,
                            ignore_geometry = True,
                            endian = 'big') as f:
    # print(np.shape(f.trace.raw[:]))
    # plot_rawtraces(f.trace.raw[:])
    
    print('\n_____SEG-Y FILE____\n')
    
    print(f'Sample format: {f.format}')
    print(f'Sample count: {len(f.samples)}')
    print(f'Trace count: {f.tracecount}')
    
    
    # modes are: .trace, .header, .iline/xline, .fast/slow, .depth_slice
    # .gather, .text, .bin
    
    # 2D seismic profiles are unstructured files
    # .trace and .header are the only modes available for unstructured files
    # traces are enumerated: 0..len(f.trace)
    # .trace mode - offers raw adressing of traces as they are laid out in the file
    # Reading a trace yields a numpy ndarray, and
    # reading multiple traces yields a generator of ndarray's
    
    # .header mode - accessing items yield header objects instead of numpy ndarray
    # Headers are dict-like objects, where
    # keys are integers, seismic unix-style keys and segyio enums
    
    print(f.samples)
    print(f.trace)
    print(f.trace[0])
    # print(f'{f.bin}')
    # print(f.text[0])
    # print(f'DELAY (109 byte): {f.header[0]['109']}')
    # print(len(f.header))
    # print(len(f.trace))
    # print('\n')
    # print(f.trace)
    # print(f.header)
    # print(len(f.samples))
    # print(type(np.fromiter(f.trace[:])))
    # print(np.fromiter(f.trace[:], dtype=np.dtype((float, len(f.samples)))))
    # plot_rawtraces(np.fromiter(f.trace[:], dtype=np.dtype((float, len(f.samples)))))


_____SEG-Y FILE____

Sample format: 4-byte IEEE float
Sample count: 320
Trace count: 3053
[19.    19.125 19.25  19.375 19.5   19.625 19.75  19.875 20.    20.125
 20.25  20.375 20.5   20.625 20.75  20.875 21.    21.125 21.25  21.375
 21.5   21.625 21.75  21.875 22.    22.125 22.25  22.375 22.5   22.625
 22.75  22.875 23.    23.125 23.25  23.375 23.5   23.625 23.75  23.875
 24.    24.125 24.25  24.375 24.5   24.625 24.75  24.875 25.    25.125
 25.25  25.375 25.5   25.625 25.75  25.875 26.    26.125 26.25  26.375
 26.5   26.625 26.75  26.875 27.    27.125 27.25  27.375 27.5   27.625
 27.75  27.875 28.    28.125 28.25  28.375 28.5   28.625 28.75  28.875
 29.    29.125 29.25  29.375 29.5   29.625 29.75  29.875 30.    30.125
 30.25  30.375 30.5   30.625 30.75  30.875 31.    31.125 31.25  31.375
 31.5   31.625 31.75  31.875 32.    32.125 32.25  32.375 32.5   32.625
 32.75  32.875 33.    33.125 33.25  33.375 33.5   33.625 33.75  33.875
 34.    34.125 34.25  34.375 34.5   34.625 34.75  34.875 